In [27]:
import sklearn
import xgboost as xgb
import joblib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
cancer=load_breast_cancer()

X_train, X_test, y_train, y_test=sklearn.model_selection.train_test_split(cancer.data, 
                                                                         cancer.target, 
                                                                         random_state=0)
xg_clas=xgb.XGBClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)
print(xg_clas.score(X_train, y_train),"\n", xg_clas.score(X_test, y_test))

1.0 
 0.9790209790209791


In [51]:
def treeOutput(obj, treeIndex, Ex):
    dump_list=obj.get_booster().get_dump()
    num_t=len(dump_list)
    preds=[]
    if(treeIndex==0):
        preds.append(obj.predict(X_test[Ex].reshape(1, -1), ntree_limit=treeIndex+1, output_margin=True))
        answer= preds[0]
        if(answer>0):
            return 1
        elif(answer<=0):
            return 0
    elif(treeIndex>0 and treeIndex<num_t):
        preds.append(obj.predict(X_test[Ex].reshape(1, -1), ntree_limit=treeIndex+1, output_margin=True))
        preds.append(obj.predict(X_test[Ex].reshape(1, -1), ntree_limit=treeIndex, output_margin=True))
        answer=preds[0]-preds[1]
        if(answer>0):
            return 1
        elif(answer<=0):
            return 0
    else:
        print("Tree Index Out of Range")

In [52]:
dump_list=xg_clas.get_booster().get_dump()
num_t=len(dump_list)

num_ex=X_test.shape[0]

m_matrix=np.zeros([num_ex, num_t])
print(m_matrix.shape)

(143, 100)


In [53]:
for i in range(num_t):
    for j in range(num_ex):
        m_matrix[j][i]=treeOutput(xg_clas, i, j)

In [55]:
print(m_matrix)

[[0. 0. 0. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 0. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]]


In [66]:
r_matrix=np.zeros([num_ex])
r_matrix.reshape(1, -1)
for i in range(num_ex):
    r_matrix[i]=xg_clas.predict(X_test[i].reshape(1, -1))
print(r_matrix)

[0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1.
 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0.]


In [88]:
f_matrix=np.empty([num_ex, num_t], dtype=object)
for i in range(num_ex):
    result=r_matrix[i]
    for j in range(num_t):
        if(m_matrix[i][j]==result):
            f_matrix[i][j]="YES"
        else:
            f_matrix[i][j]="NO"
print(f_matrix)

[['YES' 'YES' 'YES' ... 'NO' 'YES' 'YES']
 ['YES' 'YES' 'YES' ... 'NO' 'YES' 'YES']
 ['YES' 'YES' 'YES' ... 'YES' 'YES' 'YES']
 ...
 ['YES' 'YES' 'YES' ... 'YES' 'YES' 'YES']
 ['YES' 'YES' 'YES' ... 'NO' 'YES' 'YES']
 ['YES' 'YES' 'YES' ... 'YES' 'YES' 'YES']]
